In [1]:
from utils import *
import random

In [ ]:
data = json.load(open("synthetic_data/1-4.json"))

In [ ]:
data = json.load(open("synthetic_data/1-6.json"))

In [ ]:
line = data[11]
actual_answer = gpt4o_generate(line['Q'], is_print=False)
is_correct = LLM_judge(line['Q'], line['A'], actual_answer, is_print=True)
# score = LLM_score(line['Q'], line['A'], actual_answer, is_print=True)

In [2]:
import os
from os import path as osp

data_files = [osp.join("synthetic_data", f) for f in os.listdir("synthetic_data") if f.endswith(".json")]

data = [json.load(open(f)) for f in data_files]

In [6]:
count = 0
for _ in data:
    print(len(_))
    count += len(_)


19347
191
13083
907
7494
3705
881
515
167
191
3422
19347


In [5]:
count / len(data)

5770.833333333333